# Integrating the solarsystem

Remember to `pip install rebound` first.

In [1]:
%matplotlib widget
import rebound
import numpy as np
import matplotlib.pyplot as plt

print(rebound.__version__)

3.17.3


### Add bodies

In [17]:
sim = rebound.Simulation()

#Add the large planets of the solar system and the sun
sim.add("Sun")
sim.add("Jupiter")
sim.add("Saturn")

#add a the 67P/Churyumov–Gerasimenko comet
sim.add("Churyumov-Gerasimenko", m=5e-18)


Searching NASA Horizons for 'Sun'... Found: Target body name: Sun (10).
Searching NASA Horizons for 'Jupiter'... Found: Target body name: Jupiter Barycenter (5).
Searching NASA Horizons for 'Saturn'... Found: Target body name: Saturn Barycenter (6).
Searching NASA Horizons for 'Churyumov-Gerasimenko'... Found: Target body name: 67P/Churyumov-Gerasimenko.


### Plot starting orbits

In [3]:
fig = rebound.OrbitPlot(sim, unitlabel="[AU]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### What are orbits?

<img style="width: 60%" src="files/Ellipse-def0.svg.png">

### Integrating the orbit

In [19]:
#Move to center of momentum frame and set time step
sim.move_to_com()
sim.dt = 0.001

#Default rebound units are G=1, so a year is 2pi
times = np.linspace(0, 100*2*np.pi, 10000)

a = np.empty((len(times),), dtype=np.float64)
e = np.empty((len(times),), dtype=np.float64)
d = np.empty((len(times),), dtype=np.float64)
xy = np.empty((len(times),2), dtype=np.float64)

comet = sim.particles[-1] #this is a pointer to 67P
sun = sim.particles[0] #this is a pointer to the Sun
jup = sim.particles[1] #this is a pointer to Jupiter

#integrate and save orbital elements
for i, t in enumerate(times):
    sim.integrate(t)
    
    orbit = comet.calculate_orbit(primary=sun)
    a[i] = orbit.a
    e[i] = orbit.e
    xy[i,0] = comet.x
    xy[i,1] = comet.y
    d[i] = np.linalg.norm(np.array(jup.xyz) - np.array(comet.xyz))

### Plotting the integration

In [22]:
import matplotlib.gridspec as gridspec

fig = plt.figure(tight_layout=True)
gs = gridspec.GridSpec(3, 2)
ax = [
    fig.add_subplot(gs[0, 0]),
    fig.add_subplot(gs[1, 0]),
    fig.add_subplot(gs[2, 0]),
    fig.add_subplot(gs[:, 1]),
]

ax[0].plot(times/(2*np.pi), a)
ax[0].set_xlabel('Time [y]')
ax[0].set_ylabel('Semi-major axis [AU]')

ax[1].plot(times/(2*np.pi), e)
ax[1].set_xlabel('Time [y]')
ax[1].set_ylabel('Eccentricity [1]')

ax[2].plot(times/(2*np.pi), np.log10(d))
ax[2].set_xlabel('Time [y]')
ax[2].set_ylabel('Jupiter Distance [log10(AU)]')

ax[3].plot(xy[:,0], xy[:,1], '-', alpha=1, linewidth=0.1)
ax[3].set_xlabel('X [AU]')
ax[3].set_ylabel('Y [AU]')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …